In [1]:
import pandas as pd
from Bio import SeqIO
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

# Parse GTF and group exons by gene
# gtf = pd.read_csv('data/Homo_sapiens.GRCh38.113.chr.gtf.gz', sep='\t', comment='#', header=None,
#                   names=['chr', 'source', 'feature', 'start', 'end', 'score',
#                          'strand', 'frame', 'attributes'])
# exons = gtf[gtf['feature'] == 'exon']
# exons = gtf[gtf['chr'] == '21']
# exons['gene_id'] = exons['attributes'].str.extract('gene_id "([^"]+)"')
# genes = exons.groupby('gene_id')


2025-05-02 23:48:52.383860: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-02 23:48:52.384284: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-02 23:48:52.386835: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-02 23:48:52.392647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746222532.401690 2839058 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746222532.40

In [26]:
# Configuration
WINDOW_SIZE = 101
CHROMOSOME_PREFIX = ""  # "chr" if needed to match FASTA
RANDOM_SEED = 42
LEARNING_RATE = 0.0005
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [3]:
# Split genes into train/test
# gene_ids = list(genes.groups.key())
# train_genes, test_genes = train_test_split(gene_ids, test_size=0.2, random_state=42)

In [4]:
def parse_gtf(gtf_path):
    """Parse GTF file and return exon positions grouped by gene"""
    gtf = pd.read_csv(
        gtf_path,
        sep='\t',
        comment='#',
        header=None,
        names=[
            'chr', 'source', 'feature', 'start', 'end',
            'score', 'strand', 'frame', 'attributes'
        ]
    )
    
    # Filter exons and extract gene IDs
    exons = gtf[gtf['feature'] == 'exon'].copy()
    exons['gene_id'] = exons['attributes'].str.extract(r'gene_id "([^"]+)"')
    exons['chr'] = CHROMOSOME_PREFIX + exons['chr'].astype(str)
    
    # Group exons by gene
    return exons.groupby('gene_id')

In [5]:
def generate_samples_for_gene(gene_data, chromosomes, window_size=101):
    """Generate samples for a single gene"""
    sequences, labels = [], []
    half_window = window_size // 2
    chr_name = gene_data['chr'].iloc[0]
    
    if chr_name not in chromosomes:
        return [], []
    
    chr_seq = str(chromosomes[chr_name].seq)
    exon_starts = set(gene_data['start'].values)
    gene_start = gene_data['start'].min()
    gene_end = gene_data['end'].max()

    # Positive samples
    for start_pos in exon_starts:
        window_start = start_pos - half_window - 1
        window_end = window_start + window_size
        
        if 0 <= window_start < len(chr_seq) and window_end <= len(chr_seq):
            seq = chr_seq[window_start:window_end]
            sequences.append(one_hot_encode(seq))
            labels.append(1)

    # Negative samples
    num_neg = len(exon_starts)
    attempts = 0
    max_attempts = num_neg * 10
    
    while len(labels) - sum(labels) < num_neg and attempts < max_attempts:
        rand_pos = np.random.randint(gene_start, gene_end)
        if rand_pos not in exon_starts:
            window_start = rand_pos - half_window - 1
            window_end = window_start + window_size
            
            if 0 <= window_start < len(chr_seq) and window_end <= len(chr_seq):
                seq = chr_seq[window_start:window_end]
                sequences.append(one_hot_encode(seq))
                labels.append(0)
        attempts += 1

    return sequences, labels

def one_hot_encode(seq):
    mapping = {'A': [1,0,0,0], 'T': [0,1,0,0],
               'C': [0,0,1,0], 'G': [0,0,0,1]}
    return [mapping.get(nuc, [0,0,0,0]) for nuc in seq]

In [ ]:
# 3. Model Architecture
# def build_model(input_shape):
#     model = tf.keras.Sequential([
#         tf.keras.layers.InputLayer(input_shape=input_shape),
#         tf.keras.layers.Conv1D(64, 10, activation='relu'),
#         tf.keras.layers.MaxPooling1D(2),
#         tf.keras.layers.Conv1D(128, 5, activation='relu'),
#         tf.keras.layers.GlobalMaxPooling1D(),
#         tf.keras.layers.Dense(64, activation='relu'),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(1, activation='sigmoid')
#     ])
    
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(0.001),
#         loss='binary_crossentropy',
#         metrics=['accuracy', 
#                  tf.keras.metrics.Precision(name='precision'),
#                  tf.keras.metrics.Recall(name='recall')]
#     )
#     return model
# def build_model(input_shape):
#     inputs = tf.keras.Input(shape=input_shape)

#     x = tf.keras.layers.Conv1D(64, kernel_size=15, padding='valid', activation='relu')(inputs)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)

#     x = tf.keras.layers.Conv1D(128, kernel_size=5, padding='valid', activation='relu')(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)

#     x = tf.keras.layers.Conv1D(256, kernel_size=3, padding='valid', activation='relu')(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     # attention section
#     attn = tf.keras.layers.Conv1D(1, kernel_size=1)(x)
#     attn = tf.keras.layers.Softmax(axis=1)(attn)
#     x = tf.keras.layers.Multiply()([x, attn])
#     x = tf.keras.layers.GlobalMaxPooling1D()(x)

#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     x = tf.keras.layers.Dropout(0.4)(x)

#     outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

#     model = tf.keras.Model(inputs=inputs, outputs=outputs)

#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#         loss='binary_crossentropy',
#         metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
#     )

#     return model

In [27]:
def build_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    x = layers.Conv1D(64, kernel_size=10, activation='relu')(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(128, kernel_size=5, activation='relu')(x)

    # Attention layer
    x = layers.LayerNormalization()(x)
    x = layers.MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
    x = layers.GlobalAveragePooling1D()(x)

    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    # lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    #     initial_learning_rate=0.0003,
    #     decay_steps=10000
    # )
    
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )
    return model

In [18]:
# Load data
print("Loading data...")

chromosomes = SeqIO.to_dict(SeqIO.parse('data/Homo_sapiens.GRCh38.dna.chromosome.21.fa', 'fasta'))

gene_groups = parse_gtf("data/Homo_sapiens.GRCh38.113.chr.gtf.gz")

Loading data...


/tmp/ipykernel_2839058/3023959441.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gtf = pd.read_csv(


In [ ]:
# Split genes
gene_ids = list(gene_groups.groups.keys())
train_genes, test_genes = train_test_split(
    gene_ids, test_size=0.2, random_state=RANDOM_SEED
)

# Generate samples
print("Processing training data...")
X_train, y_train = [], []
for gene_id in tqdm(train_genes):
    try:
        gene_data = gene_groups.get_group(gene_id)
        seqs, lbls = generate_samples_for_gene(gene_data, chromosomes)
        X_train.extend(seqs)
        y_train.extend(lbls)
    except KeyError:
        continue

print("Processing test data...")
X_test, y_test = [], []
for gene_id in tqdm(test_genes):
    try:
        gene_data = gene_groups.get_group(gene_id)
        seqs, lbls = generate_samples_for_gene(gene_data, chromosomes)
        X_test.extend(seqs)
        y_test.extend(lbls)
    except KeyError:
        continue
# Convert to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

Loading data...


/tmp/ipykernel_2839058/3023959441.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gtf = pd.read_csv(


Processing training data...


100%|█████████████████████████████████████████████████████████████████████████████████████████| 62979/62979 [09:06<00:00, 115.30it/s]


Processing test data...


100%|█████████████████████████████████████████████████████████████████████████████████████████| 15745/15745 [02:20<00:00, 112.30it/s]


# Note
The problem was to load and process data without memory blow up. I tried to speed things up (1st chromosome took too long in comparison with 21st, the shortest one) .

In [17]:

def preprocess_data(gene_ids, gene_groups, chromosomes, window_size):
    """Preprocess all data into memory before training."""
    X_data, y_data = [], []
    for gene_id in tqdm(gene_ids):
        try:
            gene_data = gene_groups.get_group(gene_id)
            seqs, lbls = generate_samples_for_gene(gene_data, chromosomes, window_size)
            if seqs:
                X_data.extend(seqs)
                y_data.extend(lbls)
        except KeyError:
            continue
    
    X_data = np.array(X_data, dtype=np.uint8)  # Store sequences
    y_data = np.array(y_data, dtype=np.uint8)  # Store labels
    
    return X_data, y_data

# Load and preprocess all data before training
X_train, y_train = preprocess_data(train_genes, gene_groups, chromosomes, WINDOW_SIZE)
X_test, y_test = preprocess_data(test_genes, gene_groups, chromosomes, WINDOW_SIZE)

  3%|██▍                                                                                       | 1703/62979 [00:14<08:37, 118.43it/s]


KeyboardInterrupt: 

In [48]:
with h5py.File(f'genes.h5', 'w') as f:
    # Save the tuple as a dataset
    # f.create_dataset('gene_groups', data=gene_groups)
    f.create_dataset('train_genes', data=train_genes)
    f.create_dataset('test_genes', data=test_genes)
    gene_groups.obj.to_hdf('genes.h5', key='gene_df', mode='a')

HDF5ExtError: HDF5 error back trace

  File "H5F.c", line 836, in H5Fopen
    unable to synchronously open file
  File "H5F.c", line 796, in H5F__open_api_common
    unable to open file
  File "H5VLcallback.c", line 3863, in H5VL_file_open
    open failed
  File "H5VLcallback.c", line 3675, in H5VL__file_open
    open failed
  File "H5VLnative_file.c", line 128, in H5VL__native_file_open
    unable to open file
  File "H5Fint.c", line 1910, in H5F_open
    unable to lock the file
  File "H5FD.c", line 2412, in H5FD_lock
    driver lock request failed
  File "H5FDsec2.c", line 941, in H5FD__sec2_lock
    unable to lock file, errno = 11, error message = 'Resource temporarily unavailable'

End of HDF5 error back trace

Unable to open/create file 'genes.h5'

In [43]:
import h5py
import os

print("loading gtf...") 
gene_groups = parse_gtf("data/Homo_sapiens.GRCh38.113.chr.gtf.gz") # must be outside loop

# Split genes
print("splitting_genes...")
gene_ids = list(gene_groups.groups.keys())
train_genes, test_genes = train_test_split(
    gene_ids, test_size=0.2, random_state=RANDOM_SEED
)

loading gtf...


/tmp/ipykernel_2839058/3023959441.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gtf = pd.read_csv(


splitting_genes...


In [44]:

model = build_model((WINDOW_SIZE, 4))

chromosome_names = [i for i in range(1,23)] + ["X", "Y", "MT"]

np.random.shuffle(chromosome_names)

chromosome_ordinal_num = 0

for i in chromosome_names:
    filepath = f"data/bin/train_test_data{i}.h5" 

    print(f"Processing chromosome {i}...")

    if not os.path.exists(filepath):
        print(f"Chromosome {i} was not processed before. Processing...")
    
        # 2. load chromosome data
        print(f"Loading data for chromosome {i}....")
        chromosomes = SeqIO.to_dict(SeqIO.parse(f'data/Homo_sapiens.GRCh38.dna.chromosome.{i}.fa', 'fasta'))
        
        # 3. parse gtf for that chromosome
        
        # 4. preprocess train/test data
        print("preprocessing train/test data.....")
        X_train, y_train = preprocess_data(train_genes, gene_groups, chromosomes, WINDOW_SIZE)
        X_test, y_test = preprocess_data(test_genes, gene_groups, chromosomes, WINDOW_SIZE)
        
        # 5. save train/test data with h5
        with h5py.File(f'data/bin/train_test_data{i}.h5', 'w') as f:
            # Save the tuple as a dataset
            f.create_dataset('X_train', data=X_train)
            f.create_dataset('y_train', data=y_train)
            f.create_dataset('X_test', data=X_test)
            f.create_dataset('y_test', data=y_test)
    else:
        print(f"This chromosome ({i}) has already been processed. Loading train/test data....")
        try:
            with h5py.File(filepath, 'r') as f:
                X_train = f['X_train'][:]
                y_train = f['y_train'][:]
                X_test = f['X_test'][:]
                y_test = f['y_test'][:]
        except FileNotFoundError:
            print(f"Error: File {file_path} not found")
        except KeyError as e:
            print(f"Error: Dataset {str(e)} not found in the file")
    
    # 6. Train model on this chromosome
    print(f"train model on chomosome {i}.....")
    history = model.fit(
        X_train, y_train,
        epochs=35,
        batch_size=256,
        validation_split=0.1,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='val_accuracy',
                patience=3, 
                restore_best_weights=True),
            # tf.keras.callbacks.ReduceLROnPlateau(
            #     monitor='val_loss',
            #     factor=0.5,
            #     patience=2
            # )
        ]
    )
    
    
    # 7. save checkpoint model
    model.save(f"models/checkpoint_chr_{chromosome_ordinal_num}.h5")
    chromosome_ordinal_num += 1
    
    print("\nFinal evaluation:")
    model.evaluate(X_test, y_test, verbose=2)

Processing chromosome X...
This chromosome (X) has already been processed. Loading train/test data....
train model on chomosome X.....
Epoch 1/35
120/120 ━━━━━━━━━━━━━━━━━━━━ 17s 121ms/step - accuracy: 0.5423 - loss: 0.6883 - precision_12: 0.5413 - recall_12: 0.5352 - val_accuracy: 0.6174 - val_loss: 0.6548 - val_precision_12: 0.5982 - val_recall_12: 0.7147
Epoch 2/35
120/120 ━━━━━━━━━━━━━━━━━━━━ 13s 108ms/step - accuracy: 0.6176 - loss: 0.6538 - precision_12: 0.6121 - recall_12: 0.6422 - val_accuracy: 0.6815 - val_loss: 0.6043 - val_precision_12: 0.7031 - val_recall_12: 0.6282
Epoch 3/35
120/120 ━━━━━━━━━━━━━━━━━━━━ 13s 107ms/step - accuracy: 0.6786 - loss: 0.6039 - precision_12: 0.6876 - recall_12: 0.6539 - val_accuracy: 0.6876 - val_loss: 0.5901 - val_precision_12: 0.7218 - val_recall_12: 0.6106
Epoch 4/35
120/120 ━━━━━━━━━━━━━━━━━━━━ 12s 99ms/step - accuracy: 0.7086 - loss: 0.5726 - precision_12: 0.7128 - recall_12: 0.6977 - val_accuracy: 0.6824 - val_loss: 0.5969 - val_precision_1


Final evaluation:
260/260 - 2s - 8ms/step - accuracy: 0.6924 - loss: 0.5912 - precision_12: 0.7171 - recall_12: 0.6353
Processing chromosome 20...
This chromosome (20) has already been processed. Loading train/test data....
train model on chomosome 20.....
Epoch 1/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 9s 101ms/step - accuracy: 0.6997 - loss: 0.5812 - precision_12: 0.7006 - recall_12: 0.6920 - val_accuracy: 0.6959 - val_loss: 0.5831 - val_precision_12: 0.6994 - val_recall_12: 0.6703
Epoch 2/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 10s 108ms/step - accuracy: 0.7212 - loss: 0.5552 - precision_12: 0.7220 - recall_12: 0.7148 - val_accuracy: 0.6986 - val_loss: 0.5809 - val_precision_12: 0.6918 - val_recall_12: 0.6994
Epoch 3/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 9s 103ms/step - accuracy: 0.7391 - loss: 0.5359 - precision_12: 0.7370 - recall_12: 0.7392 - val_accuracy: 0.6940 - val_loss: 0.5849 - val_precision_12: 0.6944 - val_recall_12: 0.6758
Epoch 4/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 9s 101ms/step - accuracy: 0.7561 - lo


Final evaluation:
226/226 - 2s - 9ms/step - accuracy: 0.7060 - loss: 0.5721 - precision_12: 0.7109 - recall_12: 0.6943
Processing chromosome 16...
This chromosome (16) has already been processed. Loading train/test data....
train model on chomosome 16.....
Epoch 1/35
167/167 ━━━━━━━━━━━━━━━━━━━━ 17s 99ms/step - accuracy: 0.6943 - loss: 0.5881 - precision_12: 0.6962 - recall_12: 0.6943 - val_accuracy: 0.7149 - val_loss: 0.5643 - val_precision_12: 0.6984 - val_recall_12: 0.7539
Epoch 2/35
167/167 ━━━━━━━━━━━━━━━━━━━━ 17s 104ms/step - accuracy: 0.7106 - loss: 0.5698 - precision_12: 0.7153 - recall_12: 0.7036 - val_accuracy: 0.7181 - val_loss: 0.5643 - val_precision_12: 0.6971 - val_recall_12: 0.7687
Epoch 3/35
167/167 ━━━━━━━━━━━━━━━━━━━━ 16s 97ms/step - accuracy: 0.7213 - loss: 0.5569 - precision_12: 0.7264 - recall_12: 0.7140 - val_accuracy: 0.7162 - val_loss: 0.5698 - val_precision_12: 0.6902 - val_recall_12: 0.7818
Epoch 4/35
167/167 ━━━━━━━━━━━━━━━━━━━━ 18s 106ms/step - accuracy: 0.


Final evaluation:
384/384 - 4s - 10ms/step - accuracy: 0.7051 - loss: 0.5696 - precision_12: 0.6879 - recall_12: 0.7510
Processing chromosome 19...
This chromosome (19) has already been processed. Loading train/test data....
train model on chomosome 19.....
Epoch 1/35
188/188 ━━━━━━━━━━━━━━━━━━━━ 19s 103ms/step - accuracy: 0.7194 - loss: 0.5588 - precision_12: 0.7229 - recall_12: 0.7091 - val_accuracy: 0.7205 - val_loss: 0.5525 - val_precision_12: 0.7040 - val_recall_12: 0.7596
Epoch 2/35
188/188 ━━━━━━━━━━━━━━━━━━━━ 18s 95ms/step - accuracy: 0.7344 - loss: 0.5405 - precision_12: 0.7385 - recall_12: 0.7237 - val_accuracy: 0.7236 - val_loss: 0.5500 - val_precision_12: 0.7158 - val_recall_12: 0.7408
Epoch 3/35
188/188 ━━━━━━━━━━━━━━━━━━━━ 20s 105ms/step - accuracy: 0.7439 - loss: 0.5302 - precision_12: 0.7491 - recall_12: 0.7314 - val_accuracy: 0.7246 - val_loss: 0.5534 - val_precision_12: 0.7187 - val_recall_12: 0.7370
Epoch 4/35
188/188 ━━━━━━━━━━━━━━━━━━━━ 18s 98ms/step - accuracy: 0


Final evaluation:
424/424 - 4s - 9ms/step - accuracy: 0.7163 - loss: 0.5659 - precision_12: 0.7107 - recall_12: 0.7295
Processing chromosome Y...
This chromosome (Y) has already been processed. Loading train/test data....
train model on chomosome Y.....
Epoch 1/35
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - accuracy: 0.6771 - loss: 0.6022 - precision_12: 0.6850 - recall_12: 0.6489 - val_accuracy: 0.7250 - val_loss: 0.5511 - val_precision_12: 0.6919 - val_recall_12: 0.8050
Epoch 2/35
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.7073 - loss: 0.5637 - precision_12: 0.7097 - recall_12: 0.6983 - val_accuracy: 0.7516 - val_loss: 0.5378 - val_precision_12: 0.7166 - val_recall_12: 0.8270
Epoch 3/35
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.7253 - loss: 0.5360 - precision_12: 0.7276 - recall_12: 0.7178 - val_accuracy: 0.7469 - val_loss: 0.5284 - val_precision_12: 0.7120 - val_recall_12: 0.8239
Epoch 4/35
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.7505 - loss: 0.51

...

train model on chomosome 7.....
Epoch 1/35
201/201 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step - accuracy: 0.7458 - loss: 0.5309 - precision_12: 0.7519 - recall_12: 0.7322 - val_accuracy: 0.7236 - val_loss: 0.5417 - val_precision_12: 0.7630 - val_recall_12: 0.6478
Epoch 2/35
201/201 ━━━━━━━━━━━━━━━━━━━━ 18s 87ms/step - accuracy: 0.7594 - loss: 0.5149 - precision_12: 0.7720 - recall_12: 0.7348 - val_accuracy: 0.7189 - val_loss: 0.5440 - val_precision_12: 0.7619 - val_recall_12: 0.6359
Epoch 3/35
201/201 ━━━━━━━━━━━━━━━━━━━━ 19s 93ms/step - accuracy: 0.7700 - loss: 0.4985 - precision_12: 0.7825 - recall_12: 0.7468 - val_accuracy: 0.7199 - val_loss: 0.5467 - val_precision_12: 0.7533 - val_recall_12: 0.6531
Epoch 4/35
201/201 ━━━━━━━━━━━━━━━━━━━━ 17s 87ms/step - accuracy: 0.7817 - loss: 0.4841 - precision_12: 0.7936 - recall_12: 0.7603 - val_accuracy: 0.7126 - val_loss: 0.5603 - val_precision_12: 0.7509 - val_recall_12: 0.6352



Final evaluation:
406/406 - 2s - 6ms/step - accuracy: 0.7302 - loss: 0.5381 - precision_12: 0.7701 - recall_12: 0.6562
Processing chromosome 12...
Chromosome 12 was not processed before. Processing...
Loading data for chromosome 12....
preprocessing train/test data.....


100%|█████████████████████████████████████████████████████████████████████████████████████████| 15745/15745 [00:40<00:00, 384.67it/s]


train model on chomosome 12.....
Epoch 1/35
206/206 ━━━━━━━━━━━━━━━━━━━━ 18s 86ms/step - accuracy: 0.7368 - loss: 0.5348 - precision_12: 0.7513 - recall_12: 0.7066 - val_accuracy: 0.7371 - val_loss: 0.5244 - val_precision_12: 0.7487 - val_recall_12: 0.7115
Epoch 2/35
206/206 ━━━━━━━━━━━━━━━━━━━━ 18s 89ms/step - accuracy: 0.7481 - loss: 0.5207 - precision_12: 0.7628 - recall_12: 0.7187 - val_accuracy: 0.7351 - val_loss: 0.5286 - val_precision_12: 0.7385 - val_recall_12: 0.7256
Epoch 3/35
206/206 ━━━━━━━━━━━━━━━━━━━━ 18s 87ms/step - accuracy: 0.7589 - loss: 0.5072 - precision_12: 0.7721 - recall_12: 0.7334 - val_accuracy: 0.7327 - val_loss: 0.5353 - val_precision_12: 0.7439 - val_recall_12: 0.7074
Epoch 4/35
206/206 ━━━━━━━━━━━━━━━━━━━━ 18s 87ms/step - accuracy: 0.7682 - loss: 0.4930 - precision_12: 0.7810 - recall_12: 0.7442 - val_accuracy: 0.7301 - val_loss: 0.5492 - val_precision_12: 0.7493 - val_recall_12: 0.6893



Final evaluation:
461/461 - 3s - 7ms/step - accuracy: 0.7331 - loss: 0.5365 - precision_12: 0.7528 - recall_12: 0.6941


In [34]:
import h5py
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 1. Preprocess All Chromosomes First =========================================
chromosome_names = [str(i) for i in range(1, 23)] + ["X", "Y"]

# Global gene split (all chromosomes)
gene_groups = parse_gtf("data/Homo_sapiens.GRCh38.113.chr.gtf.gz") 
gene_ids = list(gene_groups.groups.keys())
train_genes, test_genes = train_test_split(gene_ids, test_size=0.2, random_state=RANDOM_SEED)

# Preprocess and save all chromosomes
for chr_name in chromosome_names:
    # Load chromosome sequence
    chromosomes = SeqIO.to_dict(SeqIO.parse(
        f'data/Homo_sapiens.GRCh38.dna.chromosome.{chr_name}.fa', 'fasta'
    ))
    
    # Filter genes present on this chromosome
    chr_genes = [g for g in gene_ids if gene_groups.get_group(g)['chr'].iloc[0] == chr_name]
    chr_train = list(set(chr_genes) & set(train_genes))
    chr_test = list(set(chr_genes) & set(test_genes))
    
    # Preprocess and save
    X_train, y_train = preprocess_data(chr_train, gene_groups, chromosomes, WINDOW_SIZE)
    X_test, y_test = preprocess_data(chr_test, gene_groups, chromosomes, WINDOW_SIZE)
    
    with h5py.File(f'data/bin/train_test_data_{chr_name}.h5', 'w') as f:
        f.create_dataset('X_train', data=X_train)
        f.create_dataset('y_train', data=y_train)
        f.create_dataset('X_test', data=X_test) 

# 2. Training with Chromosome Rotation =========================================
class ChromosomeDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, chr_files, batch_size=128):
        self.chr_files = chr_files
        self.batch_size = batch_size
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.ceil(sum(
            h5py.File(f, 'r')['X_train'].shape[0] for f in self.chr_files
        ) / self.batch_size))
    
    def __getitem__(self, index):
        # Get random chromosome
        chr_file = np.random.choice(self.chr_files)
        
        with h5py.File(chr_file, 'r') as f:
            X = f['X_train']
            y = f['y_train']
            
            # Get random batch from chromosome
            start = np.random.randint(0, len(X) - self.batch_size)
            return X[start:start+self.batch_size], y[start:start+self.batch_size]
    
    def on_epoch_end(self):
        np.random.shuffle(self.chr_files)

# Initialize
all_chr_files = [f'data/bin/train_test_data_{n}.h5' for n in chromosome_names]
model = build_model((WINDOW_SIZE, 4))

# Training with chromosome rotation
history = model.fit(
    ChromosomeDataGenerator(all_chr_files),
    epochs=100,
    steps_per_epoch=1000,  # Adjust based on total data size
    validation_data=(X_test_all, y_test_all),  # Combine test sets during preprocessing
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.h5',
            save_best_only=True
        )
    ]
)

/tmp/ipykernel_2839058/3023959441.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gtf = pd.read_csv(
 34%|███████████████████████████████▏                                                            | 1921/5661 [02:55<05:42, 10.93it/s]


KeyboardInterrupt: 

In [14]:
from joblib import Parallel, delayed
from tqdm import tqdm



def load_dataset_lazy(gene_ids, gene_groups, chromosomes, window_size):
    for gene_id in tqdm(gene_ids):
        try:
            gene_data = gene_groups.get_group(gene_id)
            seqs, lbls = generate_samples_for_gene(gene_data, chromosomes, window_size)
            if seqs:
                yield np.array(seqs, dtype=np.uint8), np.array(lbls, dtype=np.uint8)
        except KeyError:
            continue

X_train_list, y_train_list = [], []
for X_part, y_part in load_dataset_lazy(train_genes, gene_groups, chromosomes, WINDOW_SIZE):
    X_train_list.append(X_part)
    y_train_list.append(y_part)

X_train = np.concatenate(X_train_list)
y_train = np.concatenate(y_train_list)



100%|████████████████████████████████████████████████████████████████████████████████████████| 62979/62979 [00:22<00:00, 2805.58it/s]


In [ ]:
print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

model = build_model((WINDOW_SIZE, 4))
history = model.fit(
    X_train, y_train,
    epochs=35,
    batch_size=256,
    validation_split=0.1,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            patience=3, 
            restore_best_weights=True)
    ]
)

# Evaluate
print("\nFinal evaluation:")
model.evaluate(X_test, y_test, verbose=2)

from sklearn.metrics import classification_report
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))

# Save model
model.save("exon_start_detector.h5")